In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

from utils.trainer_utils import TrainConfig
from flearn.trainer.fedavg import FedAvg
from flearn.trainer.fesem import FeSEM
from flearn.trainer.fedgroup import FedGroup
from flearn.trainer.ifca import IFCA

def main():
    config = TrainConfig('femnist', 'mclr', 'fedgroup')
    trainer = FedGroup(config)
    trainer.train()
    #trainer.train_locally()

main()

Num GPUs Available:  1
The dataset size: 18345, train size: 16421, test size: 1924.
The train client count: 200. The test client count: 200.
Pre-training takes 32.18267631530762s seconds
SVD takes 0.08235287666320801s seconds
EDC Matrix calculation takes 0.00773167610168457s seconds
MADC Matrix calculation takes 0.017156124114990234s seconds
Clustering takes 0.01790904998779297s seconds
Clustering Results: Counter({1: 29, 3: 20, 0: 20, 2: 16, 4: 15})
---------- Round 0 ----------
Round 0, Group 0 has 7 client.
Round 0, Group 1 has 4 client.
Round 0, Group 2 has 1 client.
Round 0, Group 3 has 6 client.
Round 0, Group 4 has 2 client.
groupbase.py Train NKS: [223.0, 831.0, 424.0, 29.0, 200.0]
Round 0, Train ACC: 0.5723,            Train Loss: 1.0036
Round 0, Group: 4, Train ACC: 0.5067,                    Train Loss: 1.0552
Round 0, Group: 3, Train ACC: 0.6739,                    Train Loss: 0.8965
Round 0, Group: 0, Train ACC: 0.467,                    Train Loss: 1.1158
Round 0, Group: 

groupbase.py Test NKS: [226, 334, 476, 203, 297]
Round 6, Test ACC: 0.6536,            Test Loss: 1.2998
Round 6, Group: 4, Test ACC: 0.8673,                    Test Loss: 0.6515
Round 6, Group: 3, Test ACC: 0.7096,                    Test Loss: 1.0106
Round 6, Group: 0, Test ACC: 0.6744,                    Test Loss: 1.6413
Round 6, Group: 2, Test ACC: 0.5764,                    Test Loss: 1.3589
Round 6, Group: 1, Test ACC: 0.4478,                    Test Loss: 1.5308
Round: 6, Training time: 4.301, Test time: 0.249,                 Inter-Group Aggregate time: 0
---------- Round 7 ----------
Round 7, Group 0 has 8 client.
Round 7, Group 1 has 5 client.
Round 7, Group 2 has 2 client.
Round 7, Group 3 has 3 client.
Round 7, Group 4 has 2 client.
groupbase.py Train NKS: [237.0, 154.0, 379.0, 86.0, 148.0]
Round 7, Train ACC: 0.8078,            Train Loss: 0.693
Round 7, Group: 4, Train ACC: 0.8101,                    Train Loss: 0.6421
Round 7, Group: 3, Train ACC: 0.8961,               

Round 13, Group 0 has 5 client.
Round 13, Group 1 has 8 client.
Round 13, Group 2 has 2 client.
Round 13, Group 3 has 4 client.
Round 13, Group 4 has 1 client.
groupbase.py Train NKS: [78.0, 184.0, 215.0, 72.0, 300.0]
Round 13, Train ACC: 0.8481,            Train Loss: 0.5814
Round 13, Group: 4, Train ACC: 0.8718,                    Train Loss: 0.5705
Round 13, Group: 3, Train ACC: 0.9402,                    Train Loss: 0.476
Round 13, Group: 0, Train ACC: 0.8651,                    Train Loss: 0.5449
Round 13, Group: 2, Train ACC: 0.7917,                    Train Loss: 0.6342
Round 13, Group: 1, Train ACC: 0.7867,                    Train Loss: 0.6625
groupbase.py Test NKS: [226, 366, 492, 257, 403]
Round 13, Test ACC: 0.7087,            Test Loss: 0.9169
Round 13, Group: 4, Test ACC: 0.8584,                    Test Loss: 0.574
Round 13, Group: 3, Test ACC: 0.7186,                    Test Loss: 0.7777
Round 13, Group: 0, Test ACC: 0.7134,                    Test Loss: 0.7404
Round 13,

groupbase.py Test NKS: [233, 370, 508, 303, 426]
Round 19, Test ACC: 0.7141,            Test Loss: 0.8607
Round 19, Group: 4, Test ACC: 0.9013,                    Test Loss: 0.4732
Round 19, Group: 3, Test ACC: 0.7135,                    Test Loss: 0.8655
Round 19, Group: 0, Test ACC: 0.7343,                    Test Loss: 0.7364
Round 19, Group: 2, Test ACC: 0.7129,                    Test Loss: 0.9539
Round 19, Group: 1, Test ACC: 0.5892,                    Test Loss: 1.1506
Round: 19, Training time: 5.81, Test time: 0.324,                 Inter-Group Aggregate time: 0
---------- Round 20 ----------
Round 20, Group 0 has 4 client.
Round 20, Group 1 has 5 client.
Round 20, Group 2 has 5 client.
Round 20, Group 3 has 3 client.
Round 20, Group 4 has 3 client.
groupbase.py Train NKS: [466.0, 192.0, 403.0, 111.0, 388.0]
Round 20, Train ACC: 0.8917,            Train Loss: 0.4706
Round 20, Group: 4, Train ACC: 0.8734,                    Train Loss: 0.4754
Round 20, Group: 3, Train ACC: 0.958

Round 26, Group 0 has 4 client.
Round 26, Group 1 has 2 client.
Round 26, Group 2 has 7 client.
Round 26, Group 3 has 5 client.
Round 26, Group 4 has 2 client.
groupbase.py Train NKS: [91.0, 348.0, 257.0, 479.0, 191.0]
Round 26, Train ACC: 0.9122,            Train Loss: 0.4406
Round 26, Group: 4, Train ACC: 0.8901,                    Train Loss: 0.3887
Round 26, Group: 3, Train ACC: 0.9224,                    Train Loss: 0.4085
Round 26, Group: 0, Train ACC: 0.8988,                    Train Loss: 0.415
Round 26, Group: 2, Train ACC: 0.9353,                    Train Loss: 0.437
Round 26, Group: 1, Train ACC: 0.8639,                    Train Loss: 0.5673
groupbase.py Test NKS: [233, 378, 519, 311, 453]
Round 26, Test ACC: 0.7307,            Test Loss: 0.8916
Round 26, Group: 4, Test ACC: 0.9013,                    Test Loss: 0.4246
Round 26, Group: 3, Test ACC: 0.7672,                    Test Loss: 0.6359
Round 26, Group: 0, Test ACC: 0.7611,                    Test Loss: 0.6313
Round 26

...

Round 265, Group 0 has 4 client.
Round 265, Group 1 has 5 client.
Round 265, Group 2 has 5 client.
Round 265, Group 3 has 2 client.
Round 265, Group 4 has 4 client.
groupbase.py Train NKS: [379.0, 86.0, 433.0, 205.0, 146.0]
Round 265, Train ACC: 0.9279,            Train Loss: 0.3129
Round 265, Group: 4, Train ACC: 0.9024,                    Train Loss: 0.378
Round 265, Group: 3, Train ACC: 0.9884,                    Train Loss: 0.1386
Round 265, Group: 0, Train ACC: 0.9261,                    Train Loss: 0.3039
Round 265, Group: 2, Train ACC: 0.9366,                    Train Loss: 0.3013
Round 265, Group: 1, Train ACC: 0.9521,                    Train Loss: 0.2893
groupbase.py Test NKS: [257, 360, 541, 434, 332]
Round 265, Test ACC: 0.8134,            Test Loss: 0.6423
Round 265, Group: 4, Test ACC: 0.8521,                    Test Loss: 0.6475
Round 265, Group: 3, Test ACC: 0.8222,                    Test Loss: 0.6273
Round 265, Group: 0, Test ACC: 0.8521,                    Test Loss:

groupbase.py Test NKS: [239, 360, 541, 456, 328]
Round 271, Test ACC: 0.8274,            Test Loss: 0.6557
Round 271, Group: 4, Test ACC: 0.8536,                    Test Loss: 0.6431
Round 271, Group: 3, Test ACC: 0.8556,                    Test Loss: 0.5617
Round 271, Group: 0, Test ACC: 0.8484,                    Test Loss: 0.6151
Round 271, Group: 2, Test ACC: 0.7982,                    Test Loss: 0.7407
Round 271, Group: 1, Test ACC: 0.7835,                    Test Loss: 0.7171
Round: 271, Training time: 5.275, Test time: 0.272,                 Inter-Group Aggregate time: 0
---------- Round 272 ----------
Client f_00162 migrate from Group 0 to Group 3
Client f_00197 migrate from Group 2 to Group 1
Round 272, Group 0 has 6 client.
Round 272, Group 1 has 4 client.
Round 272, Group 2 has 4 client.
Round 272, Group 3 has 3 client.
Round 272, Group 4 has 3 client.
groupbase.py Train NKS: [218.0, 136.0, 2067.0, 322.0, 410.0]
Round 272, Train ACC: 0.9312,            Train Loss: 0.2781
Rou

groupbase.py Test NKS: [241, 374, 525, 443, 341]
Round 277, Test ACC: 0.7578,            Test Loss: 0.868
Round 277, Group: 4, Test ACC: 0.5228,                    Test Loss: 1.3674
Round 277, Group: 3, Test ACC: 0.8449,                    Test Loss: 0.6456
Round 277, Group: 0, Test ACC: 0.8438,                    Test Loss: 0.5877
Round 277, Group: 2, Test ACC: 0.781,                    Test Loss: 0.8027
Round 277, Group: 1, Test ACC: 0.6657,                    Test Loss: 1.2754
Round: 277, Training time: 4.292, Test time: 0.292,                 Inter-Group Aggregate time: 0
---------- Round 278 ----------
Client f_00004 migrate from Group 2 to Group 3
Round 278, Group 0 has 3 client.
Round 278, Group 1 has 3 client.
Round 278, Group 2 has 5 client.
Round 278, Group 3 has 8 client.
Round 278, Group 4 has 1 client.
groupbase.py Train NKS: [54.0, 714.0, 47.0, 651.0, 98.0]
Round 278, Train ACC: 0.9412,            Train Loss: 0.2664
Round 278, Group: 4, Train ACC: 0.9259,                 

Client f_00192 migrate from Group 0 to Group 3
Client f_00119 migrate from Group 1 to Group 4
Round 284, Group 0 has 8 client.
Round 284, Group 1 has 4 client.
Round 284, Group 2 has 2 client.
Round 284, Group 3 has 3 client.
Round 284, Group 4 has 3 client.
groupbase.py Train NKS: [150.0, 98.0, 755.0, 118.0, 166.0]
Round 284, Train ACC: 0.9378,            Train Loss: 0.2939
Round 284, Group: 4, Train ACC: 0.94,                    Train Loss: 0.3283
Round 284, Group: 3, Train ACC: 0.9592,                    Train Loss: 0.2437
Round 284, Group: 0, Train ACC: 0.9364,                    Train Loss: 0.2918
Round 284, Group: 2, Train ACC: 0.9831,                    Train Loss: 0.2008
Round 284, Group: 1, Train ACC: 0.8976,                    Train Loss: 0.3678
groupbase.py Test NKS: [243, 394, 543, 428, 316]
Round 284, Test ACC: 0.7926,            Test Loss: 0.7391
Round 284, Group: 4, Test ACC: 0.8724,                    Test Loss: 0.5682
Round 284, Group: 3, Test ACC: 0.8629,             

groupbase.py Test NKS: [242, 399, 543, 428, 312]
Round 290, Test ACC: 0.8181,            Test Loss: 0.7237
Round 290, Group: 4, Test ACC: 0.8182,                    Test Loss: 0.8904
Round 290, Group: 3, Test ACC: 0.817,                    Test Loss: 0.7125
Round 290, Group: 0, Test ACC: 0.8048,                    Test Loss: 0.7472
Round 290, Group: 2, Test ACC: 0.8668,                    Test Loss: 0.5852
Round 290, Group: 1, Test ACC: 0.7756,                    Test Loss: 0.7579
Round: 290, Training time: 6.67, Test time: 0.255,                 Inter-Group Aggregate time: 0
---------- Round 291 ----------
Client f_00023 migrate from Group 3 to Group 0
Round 291, Group 0 has 4 client.
Round 291, Group 1 has 3 client.
Round 291, Group 2 has 4 client.
Round 291, Group 3 has 4 client.
Round 291, Group 4 has 5 client.
groupbase.py Train NKS: [405.0, 218.0, 290.0, 644.0, 120.0]
Round 291, Train ACC: 0.9368,            Train Loss: 0.2787
Round 291, Group: 4, Train ACC: 0.9654,              

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Reshape, Conv2D, MaxPool2D, Flatten
from tensorflow.keras import Sequential

model = Sequential()
# Input Layer
model.add(Input(shape=(784,)))
# Reshape Layer
model.add(Reshape((28, 28, 1)))
# Conv Layer
model.add(Conv2D(24, 5, padding='same', activation='relu', input_shape=(28, 28, 1)))
# MaxPool Layer
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
# Flatten Layer
model.add(Flatten())
# Dense Layer
model.add(Dense(256, 'relu'))
# Output Layer
model.add(Dense(10, 'softmax'))

#opt = tf.keras.optimizers.SGD(learning_rate=lr)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(opt, loss_fn, metrics=['accuracy'])

In [ ]:
import numpy as np
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential

model = Sequential()
# Input Layer
model.add(Input(shape=(784,)))
# Output Layer
model.add(Dense(10, 'softmax', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    
opt = tf.keras.optimizers.SGD(learning_rate=0.003)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(opt, loss_fn, metrics=['accuracy'])

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential

model = Sequential()
# Input Layer
model.add(Input(shape=(784,)))
# Output Layer
model.add(Dense(10, 'softmax', kernel_regularizer=tf.keras.regularizers.l2(0.001)))

opt = tf.keras.optimizers.SGD(learning_rate=0.01)
#opt = tf.keras.optimizers.Adam(learning_rate=lr)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(opt, loss_fn, metrics=['accuracy'])


In [ ]:
print(model.get_weights())

In [ ]:
tf.random.set_seed(100)
sess.run(model.initializer)
print(model.get_weights())